# Проект "OCR-справка"

## 1. Задание

Для облегчения работы модераторов Заказчика требуется разработать решение для распознавания изображений (фотографий) справок о донациях, оформленных по форме 405. Необходимо рассмотреть и выбрать одно из существующих решений для обнаружения и обработки отсканированных таблиц или разработать свой подход. Оценить качество этого решения на данных, предоставленных Заказчиком.

Результат работы:
- командный репозиторий с файлами (в readme.md описание задачи и краткиq итог работы);
- отчет о ходе экспериметров и выбранном подходе;
- микросервис для интеграции в систему Заказчика.

На изображении справки требуется распознать:
- полная дата донации (число, месяц, год в формате дд.мм.гггг);
- тип донации (платно или безвозмездно);
- вид донации (цельная кровь, плазма, тромбоциты, эритроциты, гранулоциты (лейкоциты));
- дополнительно:
    - центр крови;
    - город.

Условные обозначения и сокращения:
- кр/д - кроводача (то есть цельная кровь);
- пл/д - плазмадача (плазма);
- ц/д - цитосдача (тромбоциты);
- БВ - безвозмездно;
- ПЛАТ - платно.

## 2. Проработка решения

В качестве вариантов решения рассмотрено две гипотезы:
1. Можно реализовать проект с использованием существующих решений по разпознаванию таблиц.
2. Необходима разработка собственного решения по распознаванию таблиц.

Преимуществами первой гипотезы являются простота и скорость развертывания, преимуществом второй гипотезы является полная контролируемость и управляемость.

С учетом поставленных сроков и опыта команды выбрано направление по подбору и использованию существующих решений.

### 2.1 Проверка гипотезы использования существующих решений

#### 2.1.1 Использование библиотеки `pytesseract`

Достоинства библиотеки: Поддержка русского языка, актуальность, популярность, работа "из коробки", возможность сохранения результата в pandas-датафрейм.

Результата рассмотрения: Текстовая информация распознается с ошибками, даже таблицы хорошего качества не распознаются. Применение нецелесообразно.

#### 2.1.2 Использование библиотеки `img2table`

Достоинства библиотеки: Библиотека является оболочкой для развертывания систем популярных OCR-решений. Имеет направленность на распознавание таблиц и возможность сохранения результата распознавания таблиц в pandas-датафрейме и xls или csv файле.

Поддерживаемые в `img2table` OCR-решения:
- Tesseract;
- PaddleOCR;
- EasyOCR;
- Google Vision;
- AWS Textract;
- Azure Cognitive Services.

Результаты рассмотрения:
- Tesseract. Тестовая таблица (крупные ячейка, крупные четкие цифры и буквы) распознается с ошибками. Таблицы на всех примерах справок (даже имеющих хорошее качество) не распознаются.
- PaddleOCR. Тестовая таблица распознается с ошибками, таблицы со справок не распознаются.
- EasyOCR. Тестовая таблица распознается без ошибок, таблицы со справок распознаются с большими ошибками, только со справок хорошего качества.
- Google Vision, AWS Textract, Azure Cognitive Services. Все сервисы требуют регистрации и ключа аутентификации, доступны тарифные планы с некоторыми ограничениями, регистрация недоступка на жителей России (в связи с санкциями). Тестирование не проводилось.
- Для улучшения распознавания применялось предварительное вырезание таблиц из справок и простая предобработка изображений срествами библиотеки `OpenCV`. Предварительное вырезание таблиц приводит к улучшению распознавания, предобработка срествами библиотеки `OpenCV` положительного эффекта не дает.

#### 2.1.3 Вывод по результатам рассмотрения первой гипотезы (использование существующих решений)

В результате тестирования двух существующих "коробочных" решений и применения нескольких OCR (в случае использования `img2table`) получить примелемое или близкое к примелемому качество распознавания таблиц не удалось. 

В целом в сегменте OCR-продуктов подбор решений, необходимых для реализации данного проекта, ограничивается (субъективно, в порядке уменьшения влияния):
- платностью или необходимостью регистрации;
- отсутствием API (сервисы он-лайн распознавания);
- отсутствием полноценной поддержки распознавания таблиц;
- отсутствием необходимого языкового пакета.

По результатам рассмотрения и тестирования данная гипотеза отвергнута.

### 2.2 Проверка гипотезы разработки собственного решения

Понимая нецелесообразность 100% написания всего кода "с нуля", допускаем при написании кода использование стандартных общепринятых открытых библиотек.

Учитывая цели задания и опыт рассмотрения "коробочных" OCR-решений, определим следующую структуру разрабатываемого кода:
1. Загрузка изображений/фото из файлов и их предобработка.
2. Обнаружение и извлечение таблицы из изображения.
3. Распознавание содержимого ячеек и сохранение результата.
4. Сравнение результата распознавания с проверочными csv-файлами, расчет метрики.
5. Создание микросервиса для включения в систему Зказчика.



#### 2.2.1 Загрузка изображений/фото из файлов и их предобработка

Загрузка изображений реализована стандартными средствами библиотеки `Pillow` из папки с заранее сохраненными изображениями справок.

Большая часть предоставленных Заказчиком изображений/фото имеет низкое или очень плохое качество, что негативно влияет на результаты распознавания. В связи с этим было принято решение ввести этап предобработки, рассмотрены и протестированы следующие варианты:
- Предобработка с помощью функций и методов библиотек `OpenCV` (перевод в градации серого, инвертирование, двусторонняя фильтрация, адаптивный порог и т.п.).
- Использование библиотеки `jdeskew` для выравнивания изображений.
- Предобученные модели:
  - `MIRNet` для улучшения освещения на изображениях;
  - `H-DIBCO 2018` для бинаризации изображений;
  - `RRDN` для увеличения точности.
- Разработана функция для увеличения точности изображения с использованием предобученной модели `RealESRGAN`.

Большая часть методов не дала положительного эффекта и приводила к ухудшению распознавания. Самым эффективным стало применение адаптивного порога к изображению средствами библиотеки `OpenCV`.

Использование предобученной модели `RealESRGAN` дает положительный эффект для плохих изображений и отрицательный эффект для хороших изображений, поэтому данную предобработку решено добавить в состав микросервиса для запуска пользователем в ручном режиме в случае получения некачественного распознавания.

#### 2.2.2 Обнаружение и извлечение таблицы из изображения

Разработана функция для обнаружения и вырезания таблицы с использованием библиотек `Pillow` и `OpenCV`, которая после применения адаптивного порога находит на изображении границы контуров и вырезает из них самый большой (так как таблица на справке у нас всегда одна).

#### 2.2.3 Распознавание содержимого ячеек и сохранение результата

Пробовали тестировать предобученные модели: CRNN, Keras OCR, Rosetta, RCNN и MobileNet. Не получилось из-за ошибок при загрузке.

Пришли к необходимости создания пользовательской функции, которая для распознавания подключает OCR-библиотеки. Эксперементировали с сочетанием библиотек для разных фрагментов/ячеек таблицы. Лучшее качество распознавания (при не самом лучшем времени распознавания) получилось при сочетании `pytesseract` и `EasyOCR`, это сочетание включено в итоговый вариант функции распознавания.

Пользовательская функция распознавания работает на вырезанном из справки изображении таблицы, определяя границы и контура ячеек. Затем содержимое каждой ячейки распознается с помощью "быстрой" библиотеки `pytesseract`, которая лучше распознает текст. В случае неудачного распознавания или обраружения в ячейке цифр, содержимое ячейки перераспознается в помощью более медленной, но более качественно распознающей цифры библиотеки `EasyOCR`.

Сохранение результата распознавания осуществляется в датасет из 4 столбцов: Дата; Тип донации; Вид донации; Количество.  
Распознавание происходит построчно и поячеечно с ориентацией на дату распознавания, как ключевое поле для формирования записи о донации.

#### 2.2.4 Сравнение результата распознавания с проверочными csv-файлами, расчет метрики

Для оценки качества распознавания проведено тестирование на 15 изображениях справок, предоставленных Заказчиком и имеющих проверочные csv-файлы для оценки результата.

В зависимости от качества исходных изображений метрика (процент корректно распознанных ячеек) составила от 0 до 100%.

Среднее значение метрики для всего набора изображений составило %.

#### 2.2.5 Создание микросервиса для включения в систему Зказчика

Для включения в систему Заказчика разработан микросервис с использованием библиотеки `fastapi`. Сервис позволяет выбрать на диске и распознать файл с изображением справки. В случае плохого распознавания, пользователь может применить дополнительную предобработку изображения нажатием соответствующей кнопки и попробовать улучшить результат. Дополнительная предобработка осуществляется с помощью предобученной модели `RealESRGAN`.